# Parallelism Benchmarks
This notebook illustrates the effect of enabling scheduler parallel execution across backends.

In [ ]:
from quasar import Circuit, Scheduler
from quasar.cost import Backend
from quasar.planner import Planner
import time

circuit = Circuit([
    {"gate": "T", "qubits": [0]},
    {"gate": "T", "qubits": [1]},
    {"gate": "H", "qubits": [0]},
    {"gate": "H", "qubits": [1]},
])

class SleepBackend:
    def load(self, n):
        pass
    def apply_gate(self, gate, qubits, params):
        time.sleep(0.05)
    def extract_ssd(self):
        return None

scheduler_p = Scheduler(
    backends={Backend.STATEVECTOR: SleepBackend()},
    planner=Planner(),
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
    parallel_backends=[Backend.STATEVECTOR],
)
start = time.time(); scheduler_p.run(circuit); par = time.time() - start

scheduler_s = Scheduler(
    backends={Backend.STATEVECTOR: SleepBackend()},
    planner=Planner(),
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
    parallel_backends=[],
)
start = time.time(); scheduler_s.run(circuit); ser = time.time() - start
print(f"parallel {par:.3f}s vs serial {ser:.3f}s")
